### IMDB dataset having 50K movie reviews for natural language processing or Text analytics.
### This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets

SOURCE: Kaggle
link: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [160]:
import spacy
import nltk
from nltk.corpus import stopwords
from spacy.lang.en.stop_words import STOP_WORDS
from spacy import lemmatizer
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import gensim
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
import torch 
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [124]:
nlp=spacy.load('en_core_web_sm', disable=['tagger', 'parser', 'ner'])

In [125]:
tokenizer=RegexpTokenizer("(\w+\'?\w+?)")

In [126]:
sw1=stopwords.words("english")
sw2=STOP_WORDS
stop_words=set(sw1).union(sw2)

In [127]:
def tokenize(rev):
    return(tokenizer.tokenize(str(rev).lower()))

In [128]:
def remove_stop_words(rev_tokens):
    return([tok for tok in rev_tokens if tok not in stop_words])

In [129]:
def lemmatize(rev_tokens):
    result=[]
    for tok in rev_tokens:
        temp=nlp(tok)
        for tok in temp:
            result.append(tok.lemma_)
    return result

In [130]:
def preprocess_pipeline(review):
    review=tokenize(review)
    review=remove_stop_words(review)
    review=lemmatize(review)
    return review

In [131]:
df=pd.read_csv('IMDB dataset.csv')
df.head(2)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive


In [132]:
reviews=list(df['review'])
sentiments=list(df['sentiment'])

In [133]:
reviews=list(map(lambda x: preprocess_pipeline(x), reviews))

##### word embddings using word2vec from gensim

In [135]:
dimension=100 # will also be used in the RNN unit part 
model=Word2Vec(reviews, size=dimension, window=3, min_count=3, workers=4)

In [136]:
model.sg#its CBOW not skipgram

0

In [137]:
# tip: as soon as we train our model, we have to delete the model unless there is further training or updation required in the model because it consumes lots of memory
# but, to use it even after deleting the model, we can use key'd vector model that holds all the info about the embedding model
word_vec=model.wv
del(model)

In [138]:
#save the vocabulary of the model
len(word_vec.vocab)

38859

In [139]:
word_vec.similar_by_word(word="bad", topn=10)

[('terrible', 0.7415202856063843),
 ('horrible', 0.7338898181915283),
 ('awful', 0.7099471688270569),
 ('suck', 0.6869287490844727),
 ('lousy', 0.6774225831031799),
 ('good', 0.6722398996353149),
 ('lame', 0.6422511339187622),
 ('crappy', 0.6361573934555054),
 ('mediocre', 0.6220316886901855),
 ('horrid', 0.6045772433280945)]

In [140]:
word_vec.similarity("good", "be")

0.14868656

In [141]:
#now apply contextual relation with the word
#ex: king - man + woman = queen
#example: 
word_vec.most_similar(negative=["bad"], positive=["decent"], topn=5)

[('solid', 0.4193841814994812),
 ('fine', 0.3851555585861206),
 ('workmanlike', 0.37039029598236084),
 ('modest', 0.3677048087120056),
 ('creditable', 0.3613351583480835)]

In [142]:
# from gensim.models import KeyedVectors #to store the loaded pretrained model

In [143]:
# model=KeyedVectors.load_word2vec_format('pretrained embedding path in .bin format', binary=True)

### Build the model for sentiment analysis using RNN

In [144]:
#to ensure results are reproducable set these
SEED=2031
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic=True

Take in the text in the review and convert into corresponding index (dictionary) for this dictionary, key'd vector from gensim is used. Using that dictionary we'd convert all the words in our text corpus as indices and then we pass those indices through the RNN

In [145]:
def convWordInd(embedding_model, review):
    indice_rev=[]
    for word in review:
        try:
            indice_rev.append(embedding_model.vocab[word].index)
        except:
            pass
    return torch.tensor(indice_rev)

In [146]:
review_indexes=list(map(lambda x: convWordInd(word_vec, x), reviews))

Passing the data to nn as batches. When it is done so, all the sentance should have the same length For that, all the statements are padded with the padding value (len of longest statement in the batch). So, the padding value is appended to the end of the shorter statements (that is the index value that's why it is done so)

In [147]:
pad_value=len(word_vec.index2word) #used later during RNN param initialization
pad_value #this is the length of the longest review in the batch

38859

#### BUILD THE RNN MODEL

layers:
1. Embedding layer - input - indexed reviews and convert to embedded format 
2. RNN unit - input - embedded representation
3. Fully connected unit (dense)

Embedding layer :Word embeddings are already created using gensim and these pre-trained embeddings are used within our nn. For that first extract the weights that the gensim model has learned while training

RNN Model-for training and evaluation.
After that the  dense layer is declared

forward function: for data when passed onto the model, for each batch x, with size (max(len(sentances)*len(batch)))

pytorch internally converts the indexed representation which we passed to one hot encoding.

##### NOTE
In the forward pass function, batch of reviews and the len of each review (text length) is passed. But, in case of RNN, each and every review must have same length. However, it is not required that the model to read that padded values. Hence we use the pytorch inbuilt function called "packpaddedsequence". It packs the padded values automatically and internally handles for us. 

So the packed sequence is passed through the RNN and after passing through the RNN, the packed sequence is unlocked. 

In [148]:
embed_weights=torch.Tensor(word_vec.vectors)

In [149]:
class RNN(nn.Module):
    def __init__(self, inp_dim, embed_dim, hidden_dim, out_dim, embed_weights):
        super().__init__()
        self.embedding_layer=nn.Embedding.from_pretrained(embed_weights)
        self.rnn=nn.RNN(embed_dim, hidden_dim)
        self.den=nn.Linear(hidden_dim, out_dim)
    def forward(self, x, text_lens):
        #x[size= sent len, batch size].. this is the input shape
        embedded=self.embedding_layer(x) #[sent len, batch_size, embedding_size] this is the  shape of the embedding
        packed_embedded=nn.utils.rnn.pack_padded_sequence(embedded, text_lens)
        packed_out, hidden=self.rnn(packed_embedded)#[sentance len, batch_size, hidden_dim] is  the size of packed_output and for hidden state - [1, batch_size, hidden dimension]
#in case of need of the packed sequences to be unlocked:
        output, output_lens=nn.utils.rnn.pad_packed_sequence(packed_out)
#but here, only the hidden values is needed to pass through the dense layer
        #so, squeeze the layer to remove that extra dimension within the  hidden state
        return self.den(hidden.squeeze(0))

Define the parameters for the  RNN class and create the objects of the  RNN Class

In [150]:
input_dim=pad_value
emb_dim=dimension#declared above somewhere
hidden_dim=256
out_dim=1

In [151]:
model=RNN(input_dim, emb_dim, hidden_dim, out_dim, embed_weights)
model

RNN(
  (embedding_layer): Embedding(38859, 100)
  (rnn): RNN(100, 256)
  (den): Linear(in_features=256, out_features=1, bias=True)
)

In [152]:
optimizer=optim.SGD(model.parameters(), lr=0.003)
loss_function=nn.BCEWithLogitsLoss()

In [153]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

#### prepare the data for :

    a. Training

    b. Validation
    
    c. Testing

In [155]:
#binary encoding the y vals
sentiments=[0 if label == 'negative' else 1 for label in sentiments]
sentiments[:5]

[1, 1, 1, 0, 1]

In [157]:
X_train, X_test, Y_train, Y_test=train_test_split(review_indexes, sentiments, test_size=0.25)
X_train, X_val, Y_train, Y_val=train_test_split(X_train, Y_train, test_size=0.2)

In [169]:
#now batches out of entire dataset is prepared
batch_size=128
def iterate_func(x, y):
    size=len(x)
    permute=np.random.permutation(size) #for creating the random blocks from the list of reviews
    iterator=[]
    for i in range(0, size, batch_size): #from 0 to size stepping from batch size
        indices=permute[i: i+batch_size]
        batch={}
        batch["text"]=[x[i] for i in indices]
        batch["label"]=[y[i] for i in indices]
        #sort the texts based on their lengths
        batch["text"], batch["label"]=zip(*sorted(zip(batch["text"], batch["label"]), key=lambda x: len(x[0]), reverse=True))
        batch["length"]=[len(rev) for rev in batch["text"]]
        batch["length"]=torch.IntTensor(batch["length"])
        #Now, reviews are padded in each batch and are passed into the model. 
        #For padding, pytorch offers the method within its nn module
        batch["text"]=torch.nn.utils.rnn.pad_sequence(batch["text"], batch_first=True).t()#transpose is performed so that it is accepted into rnn
        batch["label"]=torch.Tensor(batch["label"])
        
        #now pushing all to the gpu
        batch["text"]=batch["text"].to(device)
        batch["label"]=batch["label"].to(device)
        batch["length"]=batch["length"].to(device)
        
        iterator.append(batch)
        
    return iterator

In [170]:
train_iter=iterate_func(X_train, Y_train)
val_iter=iterate_func(X_val, Y_val)
test_iter=iterate_func(X_test, Y_test)

In [171]:
model=model.to(device)
criterion=loss_function.to(device)

#### defining a function to know the accuracy subsequently

In [172]:
def binary_acc(preds, y):
    round_preds=torch.round(torch.sigmoid(preds))
    pos=(round_preds==y).float()
    accuracy=pos.sum()/len(pos)
    return accuracy

#### defining the function to train the model 
set the loss and accuracies as false from the previous use and set the model to training mode

In [185]:
def train(model, iterator, optimizer, criterion):
    epoch_loss=0
    epoch_accuracy=0
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions=model(batch["text"], batch["length"]).squeeze(1)
        loss=criterion(predictions, batch["label"])
        accuracy=binary_acc(predictions, batch["label"])
        loss.backward()
        optimizer.step() #updates the weight for the model
        epoch_loss+=loss.item()
        epoch_accuracy+=accuracy.item()
    
    #return the average epoch loss and iterator
    return(epoch_loss/len(iterator), epoch_accuracy/len(iterator))

#### defining the function to Test the model
set the loss and accuracies as false from the previous use and set the model to evaluation mode

In [186]:
def evaluator(model, iterator, criterion):
    epoch_loss=0
    epoch_accuracy=0
    model.eval()
    #to prevent any gradient calculation with nograd is used
    with torch.no_grad():
        for batch in iterator:
            predictions=model(batch["text"], batch["length"]).squeeze(1)
            loss=criterion(predictions, batch["label"])
            accuracy=binary_acc(predictions, batch["label"])
            epoch_loss+=loss.item()
            epoch_accuracy+=accuracy.item()

    #return the average epoch loss and iterator
    return(epoch_loss/len(iterator), epoch_accuracy/len(iterator))

#### Running the model and evaluation 

In [187]:
epochs=5

for epoch in range(epochs):
    train_loss, train_accuracy=train(model, train_iter, optimizer, criterion)
    valid_loss, valid_accuracy=evaluator(model, val_iter, criterion)
    
    print("Epoch number: ", epoch)
    print("Train Loss = ", train_loss, " Train Accuracy = ", train_accuracy)
    print("Validation Loss = ", valid_loss, " Validation Accuracy = ", valid_accuracy)

Epoch number:  0
Train Loss =  0.575541971592193  Train Accuracy =  0.7060948582405739
Validation Loss =  0.5174620601080232  Validation Accuracy =  0.7494842773776943
Epoch number:  1
Train Loss =  0.44786902539273526  Train Accuracy =  0.7992464539852548
Validation Loss =  0.412110616089934  Validation Accuracy =  0.8267381241766073
Epoch number:  2
Train Loss =  0.424059035042499  Train Accuracy =  0.818062943346957
Validation Loss =  0.4688831315202228  Validation Accuracy =  0.7866929638183723
Epoch number:  3
Train Loss =  0.4816316267277332  Train Accuracy =  0.7761303191489362
Validation Loss =  0.5920464063094835  Validation Accuracy =  0.6814855599807481
Epoch number:  4
Train Loss =  0.47332527510663297  Train Accuracy =  0.7838541667512122
Validation Loss =  0.4008926117824296  Validation Accuracy =  0.8312263044260316


In [188]:
test_loss, test_accuracy=evaluator(model, test_iter, criterion)
print("Epoch number: ", epoch)
print("Test Loss = ", test_loss, " Test Accuracy = ", test_accuracy)

Epoch number:  4
Test Loss =  0.39689796554798984  Test Accuracy =  0.8333978683364635
